### Imports

In [2]:
import datetime

import pandas as pd
import pyodbc

### Connection String

In [3]:

driver = os.environ["DB_DRIVER"]
server = os.environ["DB_SERVER"]
database = os.environ["DB_DATABASE"]
username = os.environ["DB_USERNAME"]
password = os.environ["DB_PASSWORD"]
connection_str = f"Driver={driver};Server={server},1433;Database={database};Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"


##  Load clusters from csv

In [13]:
clusters_with_title = pd.read_csv('../../data/1405_data_topic_title.csv')

In [15]:
clusters_with_title.head()

,Unnamed: 0,topic_title,cluster_number
0,0,AfD: Protest gegen das politische Establishmen...,0
1,1,Kritik an der aktuellen deutschen Regierung: F...,1
2,2,Kritik an politischer Korrektheit und Meinungs...,2
3,3,Herausforderungen der Einwanderungspolitik: Id...,3
4,4,Hohe Energiepreise und steigende Kosten: Belas...,4


##  Load cluster assignments from csv

In [5]:
cluster_assignments = pd.read_csv("../../data/1405_data_with_clusters.csv")

In [10]:
cluster_assignments.head()

,Unnamed: 0,video_id,cluster,core_message,video_playcount,video_timestamp
0,0,7005858788343319813,23,Mach so bisschen Übergang deine Haare kürzer d...,613338,1631178624
1,1,7040421798785125638,1;22;15,Kritik an ungeimpftem Personal in der Pflege;...,30603,1639225941
2,2,7052592640344329477,13,7 Millionen sozialgeldempfänger davon gut die ...,27175,1642059688
3,3,7069273153373097221,22;17;17,- Hohe Anzahl von Arztbesuchen aufgrund von Im...,235154,1645943422
4,4,7080134745832557830,16;1;0,Unterschied zwischen Gesundheitspolitik und p...,7439,1648472331


## Add Clusters and Video2Cluster link to database

In [20]:
def add_clusters_and_links_to_db(cluster_assignments, clusters_with_title, connection_str):
    """
    Add clusters and their links to videos in the SQL database.
    Args:
        cluster_assignments: DataFrame with video_id and cluster data.
        clusters_with_title: DataFrame with cluster_number and topic_title.
        connection_str: Connection string to the SQL database.
    """
    
    with pyodbc.connect(connection_str) as cnxn:
        cursor = cnxn.cursor()
        
        # Get all unique cluster ids already in the SQL database
        cursor.execute("SELECT id FROM dbo.Clusters")
        existing_cluster_ids = {row[0] for row in cursor.fetchall()}
        
        # Extract unique cluster ids and their descriptions from the DataFrame
        clusters_to_add = clusters_with_title[['cluster_number', 'topic_title']].drop_duplicates()
        
        # Determine which cluster ids need to be added
        clusters_to_add = clusters_to_add[~clusters_to_add['cluster_number'].isin(existing_cluster_ids)]
        

        # Add new cluster ids to the Clusters table
        for _, row in clusters_to_add.iterrows():
            cluster_id = int(row['cluster_number'])
            description = str(row['topic_title'])
            cursor.execute("INSERT INTO dbo.Clusters (id, description, timestamp) VALUES (?, ?, ?)", cluster_id, description, datetime.datetime.now())
        
        # Commit the new clusters
        cnxn.commit()
        
        # Insert cluster-video relationships into VideoClusters table
        for _, row in cluster_assignments.iterrows():
            video_id = int(row['video_id'])
            clusters = map(int, row['cluster'].split(';'))
            
            for cluster_id in clusters:
                # Generate a new unique id for the VideoClusters table
                cursor.execute("SELECT ISNULL(MAX(id), 0) + 1 FROM dbo.VideoClusters")
                new_id = cursor.fetchone()[0]
                
                cursor.execute("INSERT INTO dbo.VideoClusters (id, cluster_id, video_id) VALUES (?, ?, ?)", new_id, cluster_id, video_id)

        #Commit the relationships
        cnxn.commit()
        print("Clusters and relationships added to database successfully")


Clusters and relationships added to database successfully


In [ ]:
# Example usage
add_clusters_and_links_to_db(cluster_assignments, clusters_with_title, connection_str)

## Add core message to db video entries

In [ ]:
def update_videos_in_db(update_df, connection_str):
    """
    Update existing entries in the Videos table based on video_id.
    Args:
        update_df: DataFrame with video_id, core_message.
        connection_str: Connection string to the SQL database.
    """
    
    with pyodbc.connect(connection_str) as cnxn:
        cursor = cnxn.cursor()
        
        for _, row in update_df.iterrows():
            query = """
            UPDATE dbo.Videos
            SET core_messages_de = ?
            WHERE id = ?
            """
            data = (
                str(row['core_message']),
                int(row['video_id'])
            )
            cursor.execute(query, data)
        
        cnxn.commit()
        print("Videos updated successfully")

Videos updated successfully


In [ ]:
# Example usage
update_videos_in_db(cluster_assignments, connection_str)

## Get videos by cluster id

In [ ]:
def get_videos_by_cluster(cluster_id, connection_str):
    """
    Retrieve all videos based on a given cluster_id.
    Args:
        cluster_id: The cluster_id to filter videos by.
        connection_str: Connection string to the SQL database.
    Returns:
        A list of videos matching the cluster_id.
    """
    query = """
    SELECT v.*
    FROM Videos v
    JOIN VideoClusters vc ON v.id = vc.video_id
    WHERE vc.cluster_id = ?
    """

    with pyodbc.connect(connection_str) as cnxn:
        cursor = cnxn.cursor()
        cursor.execute(query, cluster_id)
        videos = cursor.fetchall()
    
    return videos


In [ ]:
# Example usage
cluster_id = 1  # Replace with your desired cluster_id
videos = get_videos_by_cluster(cluster_id, connection_str)

for video in videos:
    print(video)

## Get cluster ids for video_id

In [4]:
## Get cluster ids for video_id
def get_clusters_by_video(video_id, connection_str):
    """
    Retrieve all clusters based on a given video_id.
    Args:
        video_id: The video_id to filter clusters by.
        connection_str: Connection string to the SQL database.
    Returns:
        A list of clusters matching the video_id.
    """
    query = """
    SELECT c.*
    FROM Clusters c
    JOIN VideoClusters vc ON c.id = vc.cluster_id
    WHERE vc.video_id = ?
    """

    with pyodbc.connect(connection_str) as cnxn:
        cursor = cnxn.cursor()
        cursor.execute(query, video_id)
        clusters = cursor.fetchall()
    
    return clusters

In [6]:
clusters = get_clusters_by_video(7040421798785125638, connection_str)
print(clusters)

[(1, 'Kritik an der aktuellen deutschen Regierung: Fehlende Kompetenz, fehlerhafte Gesetzgebung und Missmanagement', datetime.datetime(2024, 5, 14, 19, 29, 5, 150000)), (22, 'Wirtschaftliche Unsicherheit und gesellschaftliche Herausforderungen: Probleme in der Arbeitswelt, fehlende Fachkräfte und finanzielle Belastungen', datetime.datetime(2024, 5, 14, 19, 29, 6, 143000)), (15, 'Forderung nach politischem Wandel und Regierungsrücktritt: Kritik an der aktuellen Regierung und Forderung nach Veränderung', datetime.datetime(2024, 5, 14, 19, 29, 5, 810000))]
